```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Static Layer](#static-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNpdT81qwzAMfhWjUwptzcZgI4dBt_RQKBssx7gHxXETscQ2tswIpe8-d91gm06Svj_pBNp1Bko4ju5DDxhYiP2bsiJXTG0f0A9iZ31iscfZhCtSh6J5fqoPC7FaCQUDs4-llM4b2yHjWrdxbUf5WuVh40m6y1Y-3N3f3rxsK6kg6x5F3RSRsSfbLw5X3yz_F10zMuk_2V_aXVOQHbAlRstR-IwqOyVLmjyOxPMvS1jCZMKE1OU_T5e1Ah7MZBSUue0wvCtQ9px5yedLzbYjdgHKI47RLAETu3q2GkoOyfyQKsJ84vTNOn8C_tRr3A)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNpdT81qwzAMfhWjUwptzcZgI4dBt_RQKBssx7gHxXETscQ2tswIpe8-d91gm06Svj_pBNp1Bko4ju5DDxhYiP2bsiJXTG0f0A9iZ31iscfZhCtSh6J5fqoPC7FaCQUDs4-llM4b2yHjWrdxbUf5WuVh40m6y1Y-3N3f3rxsK6kg6x5F3RSRsSfbLw5X3yz_F10zMuk_2V_aXVOQHbAlRstR-IwqOyVLmjyOxPMvS1jCZMKE1OU_T5e1Ah7MZBSUue0wvCtQ9px5yedLzbYjdgHKI47RLAETu3q2GkoOyfyQKsJ84vTNOn8C_tRr3A)

Required steps:

1. **[CBS](https://opendata.cbs.nl/ODataApi/odata/84721NED/TypedDataSet?$select=RegioS,%20Code_1,%20Code_42,%20Naam_43,%20Inwonertal_52)**: contains the static `"inhabitants per municipality"` data collected by CBS to be used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[CBS_INHABITANTS_PER_GM]` table.
3. **[Inhabitants Per Municipality](#static-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSSTATIC].[CBS_INHABITANTS_PER_GM]` using business logic implemented in `[dbo].[SP_INSERT_SL_CBS_INHABITANTS_PER_GM]`.

- **<font color=teal>SL</font>**: Static Layer


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Utils/MunicipalityRemapping.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[CBS_INHABITANTS_PER_GM]'))
CREATE TABLE [VWSSTAGE].[CBS_INHABITANTS_PER_GM] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [REGIONS] [VARCHAR](100),
    [CODE_1] [VARCHAR](100),
	[CODE_42] [VARCHAR](100),
	[NAME_43] [VARCHAR](100),
	[INHABITANTS_50] [VARCHAR](100)
);
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[CBS_INHABITANTS_PER_GM]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_GM] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [VRCODE] VARCHAR(5),
    [GMCODE] VARCHAR(10),
	[INHABITANTS] BIGINT
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_VC_GC_CBS_INHABITANTS_PER_GM]
	ON [VWSSTATIC].[CBS_INHABITANTS_PER_GM]
GO

CREATE INDEX [NCIX_DLI_VC_GC_CBS_INHABITANTS_PER_GM]
	ON [VWSSTATIC].[CBS_INHABITANTS_PER_GM] (
		[DATE_LAST_INSERTED],
		[VRCODE], 
		[GMCODE]
	) 
	INCLUDE (
		[INHABITANTS]
	);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_CBS_INHABITANTS_PER_GM]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_GM] (
        [VRCODE], 
        [GMCODE], 
        [INHABITANTS]
    )
    SELECT
        ISNULL(B.[VR_CODE], [CODE_42]),
        ISNULL(B.[GM_CODE_NEW], [CODE_1]),
        CAST(ROUND(SUM(CAST(A.[INHABITANTS_50] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT)
    FROM
        [VWSSTAGE].[CBS_INHABITANTS_PER_GM] A
    LEFT JOIN (
            SELECT 
                * 
            FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING] WITH(NOLOCK)
            WHERE [ACTIVE] = 1 
                AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING]  WITH(NOLOCK))  
    ) B ON A.[CODE_1] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_GM])
    GROUP BY 
        ISNULL(B.[VR_CODE],[CODE_42]), 
        ISNULL(B.[GM_CODE_NEW],[CODE_1])
END;
GO